In [2]:
import ee
ee.Initialize()

In [3]:
dataset = ee.ImageCollection('MODIS/061/MCD12Q1')
igbp_landcover = dataset.select('LC_Type1')

In [4]:
# set visibility parameters
igbp_landcover_vis = {
    'min': 1.0,
    'max': 17.0, 
    'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
    ]
}

# load Zambia boundary
zambia = ee.FeatureCollection('FAO/GAUL/2015/level0') \
            .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# clip to Zambia 
lc_zambia = igbp_landcover.map(lambda image: image.clip(zambia))

In [5]:
# define function to add layer for specific year
def add_layer(year):
    image_collection = ee.ImageCollection('MODIS/061/MCD12Q1') \
    .filter(ee.Filter.calendarRange(year, year, 'year')) \
    .select('LC_Type1')

    image = image_collection.first()
    clipped_image = image.clip(zambia)

    return(clipped_image)

In [6]:
import geemap

# loop over years
years = list(range(2001, 2021))

# display geemap
Map = geemap.Map(center=[-13, 27], zoom=6)

# add layer for each year
for year in years:
    image = add_layer(year)
    Map.addLayer(image, igbp_landcover_vis, f'Land Cover {year}')

Map

Map(center=[-13, 27], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

## District level

In [19]:
# load districts
districts = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# calculate area in hectares
districts = districts.map(lambda feature: feature.set('area_ha', feature.geometry().area().divide(1e4)))

# filter to exclude tiny districts
filtered_districts = districts.filter(ee.Filter.gte('area_ha', 5000))

In [20]:
years = list(range(2001, 2021))

# create empty list to store results
results = []

# loop over years
for year in years:
    image = ee.ImageCollection('MODIS/061/MCD12Q1') \
    .filter(ee.Filter.calendarRange(year, year, 'year')) \
    .select('LC_Type1') \
    .first()

    # loop over provinces
    def compute_area(district):
        district_name = district.get('ADM2_NAME')
        # clip land cover to district
        lc_clip = image.clip(district.geometry())
        # calculate area per class
        area_image = ee.Image.pixelArea().addBands(lc_clip)
        # grouped reduction
        reducer = ee.Reducer.sum().group(
            groupField = 1,
            groupName = 'landcover_class'
        )
        stats = area_image.reduceRegion(
            reducer = reducer,
            geometry = district.geometry(),
            scale = 500,
            maxPixels = 1e13
        )
        groups = ee.List(stats.get('groups'))

        def format_result(group):
            class_number = ee.Number(ee.Dictionary(group).get('landcover_class'))
            area = ee.Number(ee.Dictionary(group).get('sum'))
            return ee.Feature(None, {
                'year': year,
                'district': district_name,
                'landcover_class': class_number,
                'area_m2': area
            })

        formatted = groups.map(format_result)
        return ee.FeatureCollection(formatted)

    district_results = filtered_districts.map(compute_area).flatten()
    results.append(district_results)

# merge results
final_result = ee.FeatureCollection(results).flatten()


In [21]:
# export to CSV
task = ee.batch.Export.table.toDrive(
    collection = final_result,
    description = 'Zambia_LandCover_District',
    fileFormat = 'CSV'
)

task.start()

In [15]:
print(task.status())

{'state': 'RUNNING', 'description': 'Zambia_LandCover_Province', 'priority': 100, 'creation_timestamp_ms': 1745621075163, 'update_timestamp_ms': 1745621118611, 'start_timestamp_ms': 1745621078509, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'batch_eecu_usage_seconds': 39.296348571, 'id': '3OY326FZJGKJIWQRLVZV6UUL', 'name': 'projects/earthengine-legacy/operations/3OY326FZJGKJIWQRLVZV6UUL'}
